In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Econ 484 Dream Team'


Mounted at /content/gdrive
/content/gdrive/My Drive/Econ 484 Dream Team


In [ ]:
#import needed libraries

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [ ]:
#navigate to proper folder and pull data into pandas dataframe

%cd '/content/gdrive/My Drive/Econ 484 Dream Team'
tweets = pd.read_csv('/content/gdrive/My Drive/Econ 484 Dream Team/tweet_features.csv')
tweets.head()

/content/gdrive/.shortcut-targets-by-id/1othtLjYaMjzTu583kD2RAAz-OMGJGZuP/Econ 484 Dream Team


,able,access,accompanied,according,achieve,act,ago,agree,allow,alternative,answer,anti,approved,april,ask,attack,aviva,away,based,beautiful,believe,berlusconi,best,better,big,bolsonaro,brazil,brazilian,bring,build,business,called,campaign,candidate,candidates,care,catalonia,center,change,child,...,turn,unemployment,united,uribe,use,useful,values,victims,video,violence,votapsoe,vote,voted,voters,votes,voting,vox,want,wanted,wants,war,watch,water,way,week,welcome,went,win,women,work,workers,working,world,year,years,yes,yesterday,young,youth,populism
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False


In [ ]:
#split data into train and test sets
X = tweets.drop(columns=['populism'])
y = tweets['populism']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=24)    


In [ ]:
#check to make sure target variable is correct
print(y.value_counts())

False    1422
True      355
Name: populism, dtype: int64


In [ ]:
from sklearn.model_selection import GridSearchCV

#create grid of parameters for gridsearchcv to use
param_grid = {
    'alpha': [0.0001, 0.001, 0.005, 0.01, 0.1, 1],
    'fit_intercept': [0,1],
    'max_iter': [15, 20, 50, 100, 500, 1000]
}

#implement gridsearchCV to find proper tuning parameters for lasso model
lasso = Lasso()
grid_search_lasso = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=3)
grid_search_lasso.fit(X_train, y_train)
print("Lasso best parameters: {:}".format(grid_search_lasso.best_params_))
print("Lasso score: {:}".format(grid_search_lasso.score(X_test, y_test)))

#implement gridsearchCV to find proper tuning parameters for ridge model
from sklearn.linear_model import Ridge
ridge = Ridge()
grid_search_ridge = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=3)
grid_search_ridge.fit(X_train, y_train)
print("Ridge best parameters: {:}".format(grid_search_ridge.best_params_))
print("Ridge score: {:}".format(grid_search_ridge.score(X_test, y_test)))


In [ ]:
#implement gridsearchCV to find proper tuning parameters for elastic net model
from sklearn.linear_model import ElasticNet
alpha_grid = {'alpha': [.0001, .0005,.001, .002, .003, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ],'max_iter': [100000]}
grid_search = GridSearchCV(ElasticNet(),alpha_grid,cv=5,return_train_score=True)
five_fold=grid_search.fit(X_train,y_train)
print("5-fold ElasticNet accuracy on test set: {:.4f}".format(five_fold.score(X_test, y_test)))
print("5_fold CV best parameters: ",five_fold.best_params_)


5-fold ElasticNet accuracy on test set: 0.1104
5_fold CV best parameters:  {'alpha': 0.003, 'max_iter': 100000}


In [ ]:
from sklearn.linear_model import LassoCV
las = LassoCV().fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(las.score(X_train, y_train)))
print("Accuracy on test set: {:.4f}".format(las.score(X_test, y_test)))

Accuracy on training set: 0.255
Accuracy on test set: 0.1145


In [ ]:
#finding the most important words used in lasso classification

#print(las.coef_)
list(X.columns)
word_list = list()
best_vars = list()

for i in range(len(las.coef_)):
  if abs(las.coef_[i])>0.00001:
    word_list.append((list(X.columns)[i], round(las.coef_[i], 3)))
    best_vars.append(list(X.columns)[i])
    #print(las.coef_[i])
    #print(list(X.columns)[i])

#print(word_list)

def custom_sort(t):
    return abs(t[1])

word_list.sort(key=custom_sort, reverse=True)
print(word_list)


[('alternative', 0.057), ('best', -0.068), ('build', -0.047), ('candidate', -0.015), ('center', -0.047), ('ciro', -0.098), ('citizens', 0.017), ('close', 0.016), ('common', -0.012), ('community', -0.043), ('corrupt', 0.31), ('corruption', 0.07), ('country', -0.048), ('course', -0.041), ('defend', -0.063), ('democratic', -0.06), ('did', 0.137), ('dirty', 0.276), ('economic', -0.028), ('education', -0.027), ('election', -0.028), ('electoral', 0.093), ('equality', -0.036), ('fear', 0.064), ('follow', -0.047), ('freedom', 0.037), ('future', -0.008), ('going', 0.05), ('great', -0.002), ('haddad', -0.015), ('hand', 0.013), ('happy', -0.012), ('hazquepase', -0.039), ('help', -0.033), ('hope', -0.04), ('interview', -0.009), ('italians', 0.25), ('italy', -0.021), ('jobs', -0.029), ('join', -0.005), ('justice', 0.12), ('law', -0.031), ('leader', 0.053), ('leave', -0.015), ('let', 0.011), ('lie', 0.114), ('lies', 0.124), ('life', -0.075), ('like', 0.036), ('live', -0.076), ('love', -0.042), ('mak

In [ ]:
X_new = X[best_vars]
X_new.head()

,alternative,best,build,candidate,center,ciro,citizens,close,common,community,corrupt,corruption,country,course,defend,democratic,did,dirty,economic,education,election,electoral,equality,fear,follow,freedom,future,going,great,haddad,hand,happy,hazquepase,help,hope,interview,italians,italy,jobs,join,...,power,press,problem,program,proposal,propose,public,quality,reach,read,real,renzi,right,rights,said,salvini,sanchezcastejon,say,seen,society,spain,stop,streets,support,tax,tell,thank,thanks,think,time,torra,uribe,video,vote,voters,votes,vox,want,world,young
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
